In [ ]:
import glob 
import os
import cv2
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import glob
import os
from sklearn.metrics import confusion_matrix


# Train the model and save checkpoints

In [ ]:
!python3 -m keras_segmentation train \
 --checkpoints_path="./ch12/" \
 --train_images="data1/train/" \
 --train_annotations="data1/train_label1/" \
 --val_images="data1/test/" \
 --val_annotations="data1/test_label1/" \
 --epochs=20\
 --steps_per_epoch=350\
 --n_classes=2 \
 --input_height=256 \
 --input_width=256 \
 --model_name="ror_unet"

# Clear the output folder

In [ ]:
import glob
import os
imgs = glob.glob('./out/*.png')
for i in imgs:
    os.remove(i)
    

# Load the model and generate the masks for the test data

In [ ]:
!python3 -m keras_segmentation predict \
 --checkpoints_path="./ch12/" \
#  --check_num="18" \
 --input_path="data1/test/" \
 --output_path="./out/" 

# Calculate the mean percent of missed pixel and extras pixles  

In [ ]:
total_miss = 0
total_extra = 0
l=0
img_out = glob.glob('./out/*.png')
img_label = glob.glob('./data1/test_label1/*.png')
print(len(img_out))
for pic in range(0,len(img_out)):
    l=l+1
    miss_percent=0
    extra_pixels=0
    extra=0
    mean_erro=0
    equal=0
    #read images gt and output
    m_out=cv2.imread(img_out[pic])
    m_out=np.where(m_out>0,1,0)
    m_label=cv2.imread(img_label[pic])
    #check the result for missed and extras
    obj=np.unique(m_label, return_counts=True)[1][1] #target pixels of each image 
    background=np.unique(m_label, return_counts=True)[1][0] #background pixels of each image 
    result=np.unique(m_label-m_out,return_counts=True)
    if -1 in result[0]:
        index=np.where(result[0]==-1)
        extra_pixels=result[1][index]
        total_extra = extra_pixels + total_extra
        
    if 0 in result[0]:
        index=np.where(result[0]==0)
        equal=equal+1
        
    if 1 in result[0]:
        index=np.where(result[0]==1)
        miss_percent=result[1][index]/obj * 100
        total_miss = miss_percent + total_miss
        
print('total miss: ',total_miss/len(img_out))
print('total extra: ',np.int(total_extra/len(img_out)))

# Evaluate the segmentation by MIOU,Dice,FPR and TPR metrics 

In [ ]:
from sklearn.metrics import confusion_matrix
total_miou = 0
total_dice = 0
total_fpr = 0
total_tpr = 0
# img_out = glob.glob('/home/sara/resnet50_segnet_5epoch/*.png')
img_out = glob.glob('./out/*.png')
img_label = glob.glob('./data1/test_label1/*.png')
total = 0
for pic in range(0,len(img_out)):
   
    m_out=cv2.imread(img_out[pic])
#     print(np.unique(m_out))
    prediction=np.reshape(np.where(m_out>0,1,0),(-1))
    target=np.reshape(cv2.imread(img_label[pic]),(-1))

    cm = confusion_matrix(target,prediction)
    tn = cm[0][0] #true positive
    fp = cm[0][1] #false positive
    fn = cm[1][0] #false negative
    tp = cm[1][1] #true positive

    total_miou = 0.5*((tn/(tn+fp+tn+fn-tn)) + (tp/(fn+tp+fp+tp-tp))) + total_miou
    total_dice = ((tn/(tn+fp+tn+fn)) + (tp/(fn+tp+fp+tp))) + total_dice
    tpr = (tp/(tp + fn)) * 100
    fpr = (fp/(fp + tn)) * 100
#     print('tpr: ',tpr,'\n')
#     print('fpr: ',fpr,'\n')
#     total_fpr = ((fp/(fp + tn)) * 100) + total_fpr
#     total_tpr = ((tp/(tp + fn)) * 100) + total_tpr
    total_fpr = fpr + total_fpr
    total_tpr = tpr + total_tpr
print('total_miou:',(total_miou/len(img_out)),'\n')
print('total_dice*',(total_dice/len(img_out)),'\n')
print('total_fpr:',(total_fpr/len(img_out)),'\n')
print('total_tpr:',(total_tpr/len(img_out)),'\n')